In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import helpers.cusplot as cplt
import scipy.optimize as sco
import scipy.constants as scc
from ionoptics import geometry as geo
from ionoptics import beamline as bl
import pandas as pd

from functools import partial


## constants (SI)

In [ ]:
# general
c = 299792458
m_pr = 938.272 # MeV

# kicker
brho = 1.23 
lgap = 0.1
kick_l = 0.5
N = 20
u0 = 4*np.pi*1e-7

l = 3 # dist. kicker-septum

# charge distribution
sigma = 0.015
delta_x_max = 0.01

# septum
w = 0.005 # 5 mm thickness for DC septum (CERN paper)
# assuming infinte extend in y (valid - 100 mm beam tube > 6 sigma)

In [ ]:
# numeric integration of current fraction 
# incident on septum
d = {}

for delta_x in np.linspace(0,delta_x_max,10):
    I = 0
    for step in np.linspace(2*sigma-w/2, 2*sigma + w/2,10):
        I += np.exp(-(step-delta_x)**2/(2*sigma**2))*w/10
    
    I_norm = 1/(np.sqrt(2*np.pi)*sigma)*I
    
    d[delta_x] = I_norm

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(*zip(*sorted(d.items())))
ax.set_xlabel('delta x [m]')
ax.set_ylabel('fraction')

ax.set_title('int. fraction on {} mm septum'.format(w*1e3))

#convert spat. to curr_off
def dx_to_dI(dx):
    dThet = dx/l
    dI = brho*lgap/(u0*kick_l*N)*dThet
    return round(dI,3)

cplt.add_axis(ax, 'x', dx_to_dI, 'delta I [A]' )

plt.show()

## el. vs. magn. field

In [ ]:
# integrated B-field / E-field norm. to kick angle
M = 1 # 1:MeV, 1e-3: keV, ...

T = np.arange(0,100,10)* M# MeV
p = np.sqrt(T**2+2*T*m_pr) # MeV

mag = p*1e-3/0.3 * M

el = p/m_pr*c*p*1e-3/0.3 * M

ax = plt.gca()
ax.set_xlabel(r'$E$ [MeV]')
ax.set_xticks(T)
ax.set_ylabel(r'$BL/\Theta$ [Tm/rad]')
ax.set_title('electric vs. magnetic field')
ax1 = ax.twinx()
ax1.set_ylabel(r'$EL/\Theta$ [V/rad]')
ax1.spines['right'].set_color('r')

ax.plot(T,mag)
ax1.plot(T,el, c = 'r')
ax.legend(['int., norm. B-field'], loc = 2)
ax1.legend(['int., norm. E-field'], loc = 4)

plt.savefig('/home/marius/Jülich/Multiplexer/Plots/elvsmagn')
plt.show()

## beam tube induction - paper

In [ ]:
def B(t,f,tau,B0):
    w = 2*np.pi*f
    
    t0 = np.pi/w
    
    C1 = w*tau
    
    tleq = np.asarray([time for time in t if time <= t0])
    tgeq = np.asarray([time for time in t if time >= t0])
    
    Bleq = B0*(1/np.sqrt(1+C1**2)*np.sin(w*tleq-np.arctan(w*tau))+C1/(1+C1**2)*np.exp(-tleq/tau))
    Bgeq = B0*C1/(1+C1**2)*(1+np.exp(t0/tau))*np.exp(-tgeq/tau)
    
    return np.append(Bleq,Bgeq)

In [ ]:
sigma = 1.33e6 # S/m
a = 65e-3/2 # inner radius
d = 5e-3 # beam tube thickness

tau = scc.mu_0*sigma*a*d/2

fig,ax = plt.subplots(5,1,figsize = (5,20))

for i,f in enumerate([1e2,5e2,1e3,5e3,10e3]):

    w = 2*np.pi*f
    
    t0 = np.pi/w

    t = np.linspace(0,5*t0,1000)
    tleq = np.asarray([time for time in t if time <= t0])
    
    ax[i].plot(t,B(t,f,tau,50e-3))
    ax[i].plot(tleq,50e-3*np.sin(w*tleq))
    ax[i].text(0,0,'{:.2e}'.format(B(t,f,tau,50e-3).max()))
    ax[i].set_title('{:.1e}'.format(f))
    

PATH_TO_DATA = '../../Misc/'
FILENAME = 'beam_induction_sine_circular tube_{:.2e}_{:.2e}_{:.2e}'.format(sigma,a,d)
plt.savefig(PATH_TO_DATA + FILENAME, format = 'png', dpi=900)

In [ ]:
# from paper
t = np.linspace(0,7e-6,100)
plt.plot(t,B(t,0.125e6,1e-6,1))

## testing point-to-parallel w/ 7 quadrupoles (IBL - 1st section)

In [ ]:
# distances

# point - to - parallel: M11 = 0, M33 = 0

# l1 - L_Fe - l1 - L_Fe - l1,l3,l1 - L_Fe - l2 - L_Fe - l2 - L_Fe - l1,l3,l1 - L_Fe - l1 - L_Fe - l1

L_Fe = 0.3
l1 = 0.3
l2 = 0.5
l3 = 0.97

# l1,l3,l1
l4 = 2*l1+l3

l5 = l1+l3

#TODO: need bl functions

In [ ]:
# 7 quads Benat
k0,k1,k2,k3 = 0.286,1.697,2.442,5.275

In [ ]:
lengths = [l1,L_Fe,l1,L_Fe,l1,l3,l1,L_Fe,l2,L_Fe,l2,L_Fe,l1,l3,l1,L_Fe,l1,L_Fe,l1]
elements = [bl.drift,bl.qdf,bl.drift,bl.qf,bl.drift,
            partial(bl.dipole, L_max = 0.97, alpha = 0.67, beta_s=0.18,beta_e=0.18),
            bl.drift,bl.qdf,bl.drift,bl.qf,bl.drift,bl.qdf,bl.drift,
            partial(bl.dipole, L_max = 0.97, alpha = 0.67, beta_s=0.18,beta_e=0.18),
            bl.drift,bl.qf,bl.drift,bl.qdf,bl.drift]

In [ ]:
elements = bl.eles_to_peles(elements,bl.opt_quad_mult(elements,lengths),S=True)
bl.plot_M_vs_s(elements,lengths, figsize = (15,5))



plt.show()

In [ ]:
bl.opt_quad_mult(elements,lengths)

## calculating transp. M at NESP

In [9]:
# l1 - L_Fe - l1 - L_Fe - l1,l3,l1 - L_Fe - l2 - L_Fe - l2 - L_Fe - l1,l3 - l4 - L_Fe_N - l5 - L_Fe_N

L_Fe = 0.3
L_Fe_N = 0.225
l1 = 0.3
l2 = 0.5
l3 = 0.97

l4 = 4.8
l5 = 0.9


In [11]:
lengths = [l1,L_Fe,l1,L_Fe,l1,l3,l1,L_Fe,l2,L_Fe,l2,L_Fe,l1,l3,l4,L_Fe_N,l5,L_Fe_N]
elements = [bl.drift,
            partial(bl.q, k = 2),
            bl.drift,
            bl.drift,
            bl.drift,
            partial(bl.dipole, L_max = 0.97, alpha = 0.67, beta_s=0.18, beta_e=0.18),
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift,
            bl.drift]

beamline = [ele(length) for ele,length in zip(elements,lengths)]

print(bl.bl(beamline))

[[-6.00978524  5.54835061  0.          0.          0.          6.6652901 ]
 [-0.60642674  0.39346967  0.          0.          0.          0.66032386]
 [ 0.          0.          3.52642596  9.5368874   0.          0.        ]
 [ 0.          0.          0.15384522  0.69963317  0.          0.        ]
 [-0.07360561 -1.04111879  0.          0.          1.         12.01903941]
 [ 0.          0.          0.          0.          0.          1.        ]]


In [14]:
# x_0 from slit scan [cm] [rad]

x = np.array([0.418,0.010,0.451,0.005,0,8e-4])

## Matrix inverse BK drift

In [ ]:
def prop_twiss(trans,twiss_start,d):
    
    twiss_trans_x = np.array([[trans[0,0]**2, -2*trans[0,1]*trans[0,0], trans[0,1]**2],
                        [-trans[0,0]*trans[1,0], trans[1,1]*trans[0,0]+trans[0,1]*trans[1,0], -trans[0,1]*trans[1,1]],
                        [trans[1,0]**2, -2*trans[1,1]*trans[1,0], trans[1,1]**2]])

    twiss_trans_y = np.array([[trans[2,2]**2, -2*trans[2,3]*trans[2,2], trans[2,3]**2],
                        [-trans[2,2]*trans[3,2], trans[3,3]*trans[2,2]+trans[2,3]*trans[3,2], -trans[2,3]*trans[3,3]],
                        [trans[3,2]**2, -2*trans[3,3]*trans[3,2], trans[3,3]**2]])
    
    twiss_inv_x = np.linalg.inv(twiss_trans_x)
    twiss_inv_y = np.linalg.inv(twiss_trans_y)
    
    if d == 'forward':
        final = (np.dot(twiss_trans_x,twiss_start[0]),np.dot(twiss_trans_y,twiss_start[1]))
    elif d == 'backward':
        final = (np.dot(twiss_inv_x,twiss_start[0]),np.dot(twiss_inv_y,twiss_start[1]))
        
    return final
    

In [ ]:
Cyk_QN2 = np.array([[-1.1848, 7.3201, 0, 0, 0, 6.6384],
                 [-0.2166, 0.4942, 0, 0, 0, 0.6581],
                 [0, 0, -1.2521, 7.1530, 0, 0],
                 [0, 0, -0.2224, 0.4718, 0, 0],
                 [-0.6581, -1.5369, 0, 0, 1, 11.2616],
                 [0, 0, 0, 0, 0, 1]])

QN1_QN2 = np.array([[1, 1.125, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0],
                     [0, 0, 1, 1.125, 0, 0],
                     [0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 1, 1.0244],
                     [0, 0, 0, 0, 0, 1]])

In [ ]:
Cyk_QN2

In [ ]:
twiss_QN2 = (np.array([[50],[1.8],[2.2]]),np.array([[50],[0.2],[0.8]]))
twiss_start_slit = (np.array([[0.6],[-0.58],[2.1]]),np.array([[1.93],[0.38],[0.6]]))

In [ ]:
prop_twiss(Cyk_QN2,twiss_QN2,d='backward')

In [ ]:
prop_twiss(QN1_QN2,twiss_QN2,d='backward')

### build sigma matrix and propagate

In [ ]:
eps_x = 18.8e-6
eps_y = 13.2e-6

beta = np.array([0.6,1.93])
alpha = np.array([-0.58,0.38])
gamma = np.array([2.1,0.6])

beta_q = np.array([2,1.26])
alpha_q = np.array([1.8,0.2])
gamma_q = np.array([2.2,0.8])



In [ ]:
sig_0 = np.array([[eps_x*beta[0],-alpha[0]*eps_x,0,0,0,0],
                 [-alpha[0]*eps_x,eps_x*gamma[0],0,0,0,0],
                 [0,0,eps_y*beta[1],-alpha[1]*eps_y,0,0],
                 [0,0,-alpha[1]*eps_y,eps_y*gamma[1],0,0],
                 [0,0,0,0,0,0],
                 [0,0,0,0,0,0]])

sig_1_meas = np.array([[eps_x*beta_q[0],-alpha_q[0]*eps_x,0,0,0,0],
                 [-alpha_q[0]*eps_x,eps_x*gamma_q[0],0,0,0,0],
                 [0,0,eps_y*beta_q[1],-alpha_q[1]*eps_y,0,0],
                 [0,0,-alpha_q[1]*eps_y,eps_y*gamma_q[1],0,0],
                 [0,0,0,0,0,0],
                 [0,0,0,0,0,0]])

In [ ]:
def calc_sigma_1(R,sigma_0,sigma_26):
    
    sigma_0[1][5] = sigma_26
    
    return np.dot(R,np.dot(sigma_0,R.transpose()))

def opt_26(R,sigma_0,sigma_1_meas,sigma_26_init):
    
    sigma_0_part = partial(calc_sigma_1, R = R, sigma_0 = sigma_0)

    def residual(sigma_26):
    
        sigma_1 = sigma_0_part(sigma_26 = sigma_26)
        
        res = np.abs(sigma_1[0][0] - sigma_1_meas[0][0]) + np.abs(sigma_1[0][1] - sigma_1_meas[0][1]) + \
        np.abs(sigma_1[1][1] - sigma_1_meas[1][1]) + np.abs(sigma_1[1][0] - sigma_1_meas[1][0]) + \
        np.abs(sigma_1[2][2] - sigma_1_meas[2][2]) + np.abs(sigma_1[2][3] - sigma_1_meas[2][3]) + \
        np.abs(sigma_1[3][3] - sigma_1_meas[3][3]) + np.abs(sigma_1[3][2] - sigma_1_meas[3][2])
        
        return res
    
    return sco.minimize(residual, sigma_26_init,tol=1e-3)

 

In [ ]:
opt = opt_26(Cyk_QN2,sig_0,sig_1_meas,0)
opt.x

In [ ]:
test = calc_sigma_1(Cyk_QN2,sig_0,opt.x)


In [ ]:
test

In [ ]:
print(sig_1_meas)